# Redes bayesianas - Resident Evil

En este trabajo usaremos una red bayesiana construida por nosotros. Esta red se basa en un contexto imaginario del mundo de __Resident Evil__. Esta pensada con el objetivo de que un superviviente pueda saber cuales son sus posibilidades de mantenerse con vida teniendo en cuenta los recursos y otras variables del universo.

Para la realización de la red se ha usado la herramienta __Weka__. 
Una vez construida, se ha exportado en formato .xml y retocado para que el método de pgmpy la lea correctamente.

El grafo de la red es el que se muestra a continuación:

<img src='G_RE.png' width=700 height=463>

Para poder trabajar con redes bayesianas usaremos el paquete __pgmpy__. Lo primero que haremos será importar el paquete y las herramientas necesarias.

In [223]:
#Importamos el paquete pgmpy y las herramientras necesarias

import networkx  # Permite trabajar con grafos
import pgmpy.models as pgmm  # Modelos gráficos de probabilidad
import pgmpy.factors as pgmf  # Tablas de probabilidades condicionales y
                              # potenciales de probabilidad
import pgmpy.inference as pgmi  # Inferencia probabilística exacta

Además, para leer ficheros con extensión __.XMLBIF__ necesitaremos importar la herramienta __readerwrite.XMLBIF__ de pgmpy.

In [191]:
import pgmpy.readwrite.XMLBIF as rwBIF

reader = rwBIF.XMLBIFReader('residentevil.xml')
Modelo_residentevil = reader.get_model()

Vamos a ver los __nodos, aristas__ y __tablas__ de la red:

In [168]:
#Nodos de la red
Modelo_residentevil.nodes()

['Lugar',
 'Ciudad',
 'Horda',
 'Medicina',
 'Agua',
 'Comida',
 'Refugio',
 'Vehiculo',
 'Armas',
 'Grupo',
 'Soldados umbrella',
 'Cooperativo']

In [167]:
#Aristas de la red
Modelo_residentevil.edges()

[('Lugar', 'Ciudad'),
 ('Lugar', 'Horda'),
 ('Ciudad', 'Horda'),
 ('Ciudad', 'Medicina'),
 ('Ciudad', 'Agua'),
 ('Ciudad', 'Comida'),
 ('Ciudad', 'Refugio'),
 ('Ciudad', 'Vehiculo'),
 ('Ciudad', 'Armas'),
 ('Ciudad', 'Grupo'),
 ('Ciudad', 'Soldados umbrella'),
 ('Refugio', 'Cooperativo'),
 ('Grupo', 'Cooperativo')]

In [193]:
#CPDS de la red
Modelo_residentevil.cpds

[<TabularCPD representing P(Lugar:3) at 0x10e01b278>,
 <TabularCPD representing P(Ciudad:2 | Lugar:3) at 0x10e01bf60>,
 <TabularCPD representing P(Horda:2 | Ciudad:2, Lugar:3) at 0x10e01b080>,
 <TabularCPD representing P(Medicina:2 | Ciudad:2) at 0x10e01bd68>,
 <TabularCPD representing P(Agua:2 | Ciudad:2) at 0x10e0aa898>,
 <TabularCPD representing P(Comida:2 | Ciudad:2) at 0x10e0b8940>,
 <TabularCPD representing P(Refugio:2 | Ciudad:2) at 0x10e09d7b8>,
 <TabularCPD representing P(Vehiculo:2 | Ciudad:2) at 0x10443e550>,
 <TabularCPD representing P(Armas:2 | Ciudad:2) at 0x10dd8d748>,
 <TabularCPD representing P(Grupo:2 | Ciudad:2) at 0x10dd8d668>,
 <TabularCPD representing P(Soldados umbrella:2 | Ciudad:2) at 0x10dd8dd68>,
 <TabularCPD representing P(Cooperativo:2 | Refugio:2, Grupo:2) at 0x10dd8d710>]

Y por último, es necesario comprobar que la red no tiene errores de construcción. Para ello empleamos el método __check_model__ de pgmpy:

In [173]:
#Comprueba que el modelo no contiene errores, en caso de ser correcto devuelve True.
Modelo_residentevil.check_model()

/Users/anapareciendo/anaconda/lib/python3.6/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


True

# Eliminación de variables

Una vez tenemos la red construida y cargada, vamos a diseñar el algoritmo de __eliminación de variables__. Se han seguido los siguientes pasos:

1. Descartar las variables irrelevantes para la consulta antes de comen- zar (mostrando un mensaje con la lista de variables descartadas).
2. Obtener todos los factores iniciales a partir de la red, ignorando las variables descartadas y teniendo en cuenta las evidencias.
3. Eliminar las variables ocultas, a través de las correspondientes ope- raciones sobre los factores.
4. Multiplicar los factores finales (en caso de que haya más de uno).
5. Normalizar.

La consulta utiliza como ejemplo durante el desarrollo del algoritmo ha sido: la probabilidad de que __Refugio__ sea __verdadero__, sabiendo que __Ciudad__ y __Cooperativo__ son __verdaderos__.

Finalmente, con el fin de comprobar la correcta implementación del algoritmo, se ha usado una herramienta __VariableElimination__ incluida en pgmpy que aplica directamente dicho algoritmo.

In [180]:
#Paso 1: Función que descarta las variables irrelevantes.


def descarta(m, c, e):
    lista = c+e
    descartados = []
    padres = []
    for l in lista:
        subgrafo = Modelo_residentevil.subgraph(networkx.ancestors(Modelo_residentevil, l))
        for s in subgrafo:
            padres.append(s)
                          
    for n in m.nodes():
        if n not in lista and n not in padres:
            descartados.append(n)
                  
    return descartados  

In [181]:
#Construcción del ejemplo (probabilidad de Refugio sabiendo que Ciudad y Cooperativo) y llamada a la función descarta
consulta = []
consulta.append("Refugio")
evidencias = []
evidencias.append("Ciudad")
evidencias.append("Cooperativo")

descarta(Modelo_residentevil,consulta,evidencias)

['Horda',
 'Medicina',
 'Agua',
 'Comida',
 'Vehiculo',
 'Armas',
 'Soldados umbrella']

In [183]:
#Paso 2: Función que devuelve los factores iniciales reducidos según las evidencias.

def factores_iniciales(m,d,e):
    c = 0
    factores = {}
    for n in m.nodes():
        if n not in d:
            factores[n]= m.cpds[c].to_factor()     
        c+=1
    for k,v in factores.items():
        for i in v.scope():
            if i in e:
                v = v.reduce([(i, e[i])])
        
    return factores

In [184]:
#Llamada a la función factores_iniciales
d = descarta(Modelo_residentevil, consulta, evidencias)
e = {'Ciudad':1, 'Cooperativo':1}
factores = factores_iniciales(Modelo_residentevil,d,e)

for clave, valor in factores.items():
    print(clave)
    print(valor.scope())
    print(valor)

Lugar
['Lugar']
╒═════════╤══════════════╕
│ Lugar   │   phi(Lugar) │
╞═════════╪══════════════╡
│ Lugar_0 │       0.3000 │
├─────────┼──────────────┤
│ Lugar_1 │       0.2000 │
├─────────┼──────────────┤
│ Lugar_2 │       0.5000 │
╘═════════╧══════════════╛
Ciudad
['Lugar']
╒═════════╤══════════════╕
│ Lugar   │   phi(Lugar) │
╞═════════╪══════════════╡
│ Lugar_0 │       0.4000 │
├─────────┼──────────────┤
│ Lugar_1 │       0.8000 │
├─────────┼──────────────┤
│ Lugar_2 │       0.2000 │
╘═════════╧══════════════╛
Refugio
['Refugio']
╒═══════════╤════════════════╕
│ Refugio   │   phi(Refugio) │
╞═══════════╪════════════════╡
│ Refugio_0 │         0.4000 │
├───────────┼────────────────┤
│ Refugio_1 │         0.6000 │
╘═══════════╧════════════════╛
Grupo
['Grupo']
╒═════════╤══════════════╕
│ Grupo   │   phi(Grupo) │
╞═════════╪══════════════╡
│ Grupo_0 │       0.2000 │
├─────────┼──────────────┤
│ Grupo_1 │       0.8000 │
╘═════════╧══════════════╛
Cooperativo
['Grupo', 'Refugio']
╒═════

In [185]:
#Paso 3: Función que elimina las variables ocultas.

def eliminar_variables(factores, consultas, evidencias):
    eliminar = []
    res = {}
    for k,v in factores.items():
        if k not in consultas and k not in evidencias:
            eliminar.append(k)
    
    for e in eliminar:
        for k,v in factores.items():
            if e in v.scope() and len(v.scope()) > 1:
                phi_not = pgmf.factor_product(factores[e],v)
                phi_not.marginalize([e])
                factores[k] = phi_not
    for k,v in factores.items():
        for i in v.scope():
            if i in consultas:
                res[k] = v
        
    return res
        



In [186]:
#Llamada a la función eliminar_variables

consultas = []
consultas.append("Refugio")
evidencias = []
evidencias.append("Ciudad")
evidencias.append("Cooperativo")

d = descarta(Modelo_residentevil, consultas, evidencias)
e = {'Ciudad':1, 'Cooperativo':1}

factores = factores_iniciales(Modelo_residentevil,d,e)

prueba = eliminar_variables(factores,consultas,evidencias)
          
for k,v in prueba.items():
    print(k)
    print(v.scope())
    print(v)

Refugio
['Refugio']
╒═══════════╤════════════════╕
│ Refugio   │   phi(Refugio) │
╞═══════════╪════════════════╡
│ Refugio_0 │         0.4000 │
├───────────┼────────────────┤
│ Refugio_1 │         0.6000 │
╘═══════════╧════════════════╛
Cooperativo
['Refugio']
╒═══════════╤════════════════╕
│ Refugio   │   phi(Refugio) │
╞═══════════╪════════════════╡
│ Refugio_0 │         0.8400 │
├───────────┼────────────────┤
│ Refugio_1 │         0.9200 │
╘═══════════╧════════════════╛


In [188]:
#Paso 4 y 5: Función que multiplica y normaliza los factores restantes.

def mn_factores(factores):
    m = []
    for k,v in factores.items():
        if not not v.scope():
            m.append(v)
        
    phi = pgmf.factor_product(*m)
    phi.normalize()
    
    return phi

In [189]:
#Llamada a la función mn_factores
end = mn_factores(eliminar_variables(factores,consultas,evidencias))
print(end)

╒═══════════╤════════════════╕
│ Refugio   │   phi(Refugio) │
╞═══════════╪════════════════╡
│ Refugio_0 │         0.3784 │
├───────────┼────────────────┤
│ Refugio_1 │         0.6216 │
╘═══════════╧════════════════╛


In [211]:
#Uso de la herramienta VariableElimination para comprobar el algoritmo anteriormente implementado.

Modelo_re_ev = pgmi.VariableElimination(Modelo_residentevil)
consulta = Modelo_re_ev.query(['Refugio'], {'Ciudad': 1, 'Cooperativo': 1})
print(consulta['Refugio'])

╒═══════════╤════════════════╕
│ Refugio   │   phi(Refugio) │
╞═══════════╪════════════════╡
│ Refugio_0 │         0.3784 │
├───────────┼────────────────┤
│ Refugio_1 │         0.6216 │
╘═══════════╧════════════════╛


/Users/anapareciendo/anaconda/lib/python3.6/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


# Heurísticas para el orden de eliminación

-__Min Degree__: dado un conjunto de factores, la variable que se decide eliminar es la que está conectada con menos variables en el conjunto de factores. Decimos que una variable está conectada con otra respecto de un conjunto de factores, cuando ambas aparecen en un mismo factor del conjunto.

-__Min Fill__: dado un conjunto de factores, la variable que se decide eliminar es aquella que al eliminarse introduciría menos conexiones nuevas (conexiones en el sentido explicado en el punto anterior).

-__Min Factor__: dado un conjunto de factores, la variable que se decide eliminar es aquella que al eliminarse produciría el factor más pequeño (es decir, el que menos entradas tendría).

In [224]:
import EliminationOrder
from EliminationOrder import MinFill
